# pip

In [ ]:
! pip install SimpleITK

# Readme
1. 文件储存

  - 在Colab环境下直接执行`git clone`时,下载文件与`sample_data`同级,其绝对路径即为文件名(如`filename/`),且为暂时文件,<mark>当Colab关闭后文件丢失</mark>.
  - 若需要**长期存储**,可在关闭Colab前将其移动到Google Drive,在下次使用时再移动到Colab环境.
  - 或直接于Google Drive中访问及操作文件,则其绝对路径为`drive/MyDrive/filename/`.
  - <mark>Attentiono</mark>:为统一管理文件,以下涉及代码文件均存储于Google Drive,绝对路径为`drive/MyDrive/segmentation/`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 0 Colab准备工作

## 0.1 GPU信息输出

In [ ]:
# 连接到GPU,输出GPU信息
! /opt/bin/nvidia-smi

## 0.2 挂载Google Drive

### 0.2.1 手动代码挂载

In [ ]:
# 挂载GoggleDrive
from google.colab import drive
drive.mount('/content/gdrive')

### 0.2.2 自动挂载
左侧文件视图中选择链接Google Drive

In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/CTLungSeg")

# 展示路径下文件
!ls '/content/gdrive/My Drive/Colab Notebooks/CTLungSeg'



# 1 代码环境准备工作

## 1.1 Clone到Colab

In [ ]:
# 克隆代码
! git clone https://github.com/RiccardoBiondi/segmentation.git

In [ ]:
# 展示Colab环境下对应目录下文件
!ls segmentation/CTLungSeg

In [ ]:
# 移动到Google Drive长期保存

In [ ]:
# 展示Google Drive对应目录下文件
!ls drive/MyDrive/segmentation/CTLungSeg

## 1.2 数据集下载

### ~1.2.1 下载到<mark>Colab</mark>环境~

In [ ]:
# 数据集下载到Colab
! mkdir segmentation/Examples
! wget https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip -P segmentation/Examples
! unzip segmentation/Examples/COVID-19-CT-Seg_20cases.zip -d segmentation/Examples/COVID-19-CT

### 1.2.2 下载到<mark>Google Drive</mark>

In [ ]:
# 数据集下载
! mkdir drive/MyDrive/segmentation/Examples
! wget https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip -P drive/MyDrive/segmentation/Examples
! unzip drive/MyDrive/segmentation/Examples/COVID-19-CT-Seg_20cases.zip -d drive/MyDrive/segmentation/Examples/COVID-19-CT

# 2 lung_extraction脚本(肺部提取)

## ~2.1 lung_extraction.ps1~
Colab无法运行PowerShell?

In [ ]:
# 脚本执行: lung_extraction.ps1
# /bin/bash: line 1: segmentation/lung_extraction.ps1: Permission denied

# 赋予权限: ! chmod 777 -R segmentation/lung_extraction.ps1
! chmod 777 -R segmentation/lung_extraction.ps1

! segmentation/lung_extraction.ps1
# 产生error: /usr/bin/env: ‘pwsh\r’: No such file or directory

# 转为执行lung_extraction.sh

## 2.2 lung_extraction.sh

In [ ]:
# 安装所需package
! pip install SimpleITK lungmask

# 目录和文件准备
! mkdir drive/MyDrive/segmentation/Examples/INPUT
! mkdir drive/MyDrive/segmentation/Examples/LUNG
! mv drive/MyDrive/segmentation/Examples/COVID-19-CT/coronacases_002.nii.gz drive/MyDrive/segmentation/Examples/COVID-19-CT/coronacases_005.nii.gz drive/MyDrive/segmentation/Examples/INPUT

# 赋予权限
! chmod 777 -R drive/MyDrive/segmentation/lung_extraction.sh

# 脚本执行
! drive/MyDrive/segmentation/lung_extraction.sh drive/MyDrive/segmentation/Examples/INPUT/ drive/MyDrive/segmentation/Examples/LUNG/

## 2.3 shell运行问题汇总

### 2.3.1 No Module Nameed CTLungSeg
> 打开CTLungSeg中每个文件将`import CTLungSeg`改为`import drive.MyDrive.segmentation.segmentation.CTLungSeg`


### 2.3.2 脚本已经执行,但无法找到CTLungSeg.lung_extraction.py
```shell
Requirement already satisfied: SimpleITK in /usr/local/lib/python3.10/dist-packages (2.3.1)
mkdir: cannot create directory `segmentation/Examples/INPUT`: File exists
mkdir: cannot create directory `segmentation/Examples/LUNG`: File exists
mv: cannot stat 'segmentation/Examples/COVID-19-CT/coronacases_002.nii.gz': No such file or directory
mv: cannot stat 'segmentation/Examples/COVID-19-CT/coronacases_005.nii.gz': No such file or directory
Found 1 files to process
* Processing coronacases_002.nii.gz ...       /usr/bin/python3: Error while finding module specification for 'CTLungSeg.lung_extraction' (ModuleNotFoundError: No module named 'CTLungSeg')
```
> lung_extraction.sh脚本`line54`将`CTLungSeg.lung_extractions.sh`改为`drive.MyDrive.segmentation.CTLungSeg.lung_extraction.sh`

### 2.3.3 输出文件与LUNG目录同级,且文件名为`LUNGcronacases_002.nrrd`
```shell
segmentation/lung_extraction.sh segmentation/Examples/INPUT/ segmentation/Examples/LUNG/
```
> 执行lung_extractions.sh时的input和output参数地址需要以`/`结尾

### 2.3.4 无法找到待处理文件
```shell
Could not find: segmentation/Examples/INPUTcoronacases_002.nii.gz
```
> 同2.3.3

### 2.3.5 运行`lung_extraction.sh`无权限
```shell
Permission denied
```
> 在此之前执行`! chmod 777 -R segmentation/lung_extraction.sh`

## 2.2. 结果可视化

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import zipfile
import urllib.request
from drive.MyDrive.segmentation.CTLungSeg.utils import read_image, write_volume

#load the data
C01 = read_image(filename="drive/MyDrive/segmentation/Examples/LUNG/coronacases_002.nii.gz.nrrd")

#Data Visualization

# First of all convert the simple itk images to numpy array
A01 = sitk.GetArrayFromImage(C01)

# now flip them for a better visualization

A01 = np.flip(A01, axis=1)


plt.imshow(A01[150], cmap='gray')
plt.show()

# 3 labeling脚本(目标标记)

## 3.1 labeling.sh

In [ ]:
! pip install SimpleITK

# 目录准备
! mkdir drive/MyDrive/segmentation/Examples/OUTPUT

# 脚本权限
! chmod 777 -R drive/MyDrive/segmentation/labeling.sh

# 脚本labeling.sh执行
! drive/MyDrive/segmentation/labeling.sh drive/MyDrive/segmentation/Examples/LUNG/ drive/MyDrive/segmentation/Examples/OUTPUT/

## 3.2 结果可视化

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import zipfile
import urllib.request
from drive.MyDrive.segmentation.CTLungSeg.utils import read_image, write_volume

#load the data
C01 = read_image(filename="drive/MyDrive/segmentation/Examples/OUTPUT/coronacases_002.nrrd")

#Data Visualization

# First of all convert the simple itk images to numpy array
A01 = sitk.GetArrayFromImage(C01)

# now flip them for a better visualization

A01 = np.flip(A01, axis=1)


plt.imshow(A01[150], cmap='gray')
plt.show()